<font face="Arial" size=10 color=#000000 > Non-liner Optimization Exercise</font>

<font face="Arial" size=5 color=#000000 > 1. Import packages</font>

<font face="Arial" size=3 color=#404040  >
    
    * Use scipy.optimize.minimize package to do optimization
    * Use xlwt to store output in excel

In [1]:
import pandas as pd
import numpy as np
import xlwt
from scipy.optimize import minimize

<font face="Arial" size=5 color=#000000> 2. Read data</font>

<font face="Arial" size=3 color=#404040>
    
    * Use DataFrame to store data
    * Slice data into different subsets 

In [2]:
#import data from excel and store every sheet as a DataFrame
sheet1 = pd.read_excel('Non-linear optimization exercise.xlsx',sheet_name='Mock balance sheet',header=2)
sheet2 = pd.read_excel('Non-linear optimization exercise.xlsx',sheet_name='Linkages',header=0)

#slice data into different DataFrame
characteristics = sheet1.loc[:,'Product':'CCAR losses, actual'] #256 rows × 34 columns
constraint_capacity = sheet1.loc[:,'Shrink':'Grow'] #256 rows × 2 columns, all are constant numbers
constraint_ratio = sheet1.loc[0:28,'Ratio':'Optimized Ratio'] #29 rows × 4 columns, including NaN. To match cells in excel, keep NaN for a while
ratio_helper = sheet1.loc[0:51,'Ratio Components':'Optimized Ratio Components'] #52 rows × 4 columns,
GSIB_coefficient = sheet1.loc[0:8,'Metrics':'Basel'] #9 rows × 3 columns, all are constant numbers
linkages = sheet2 #96 rows × 90 columns，all are constant numbers

#define constant number
N = len(characteristics['Product']) #number of products in total
n1 = len(constraint_ratio.dropna(axis = 0,how = 'any',thresh = 2)) # number of ratio constraints
n2 = len(linkages.columns)-1 # number of linkage constraints

<font face="Arial" size=5 color=#000000 > 3.Define Objective function</font>

<font face="Arial" size=3 color=#404040>
    
    * Denote x as an array which stores 256 variables, namely, optimized notional 
    * To caluculate maximized sav, use minimized sav*(-1)

In [3]:
def objective(x):
    return x.dot(characteristics['NSI']) * (-1)

<font face="Arial" size=5 color=#000000 > 4.Define Constraints</font>

<font face="Arial" size=4 color=#404040 > 4.1 Constraint 1: CAPACITY</font>
<font face="Arial" size=3 color=#404040>
    
    * The optimized notional should be larger than (staring notional + shrink) and smaller than (staring notional + grow)
    * The optimized notional should be larger than 0
    * Each element in x has individual lower and upper limits. To simplify codes, set parameter "bounds", rather than 256*2 constraints

In [4]:
lower = characteristics.iloc[:,1] + constraint_capacity.iloc[:,0]
upper = characteristics.iloc[:,1] + constraint_capacity.iloc[:,1]
a = max(0,lower[0])
b = max(0,upper[0])
bounds_capacity = np.array([[a,b]])

for i in np.arange(1,N):
    a = max(0,lower[i])
    b = max(0,upper[i])
    bounds_capacity = np.append(bounds_capacity,[[a,b]], axis = 0)

<font face="Arial" size=4 color=#404040 > 4.2 Constraint 2: RATIOS</font>
<font face="Arial" size=3 color=#404040>
    
    * Optimized raios should be no less than constraints, 24 constraints
    * GAAP asset = GAAP liability, 1 constraints
    * To simplify codes, record constraints status into an array. If constrint holds, record 1, otherwise 0. Only when all contraints holds, the product of the array equals to 1, otherwise 0.

In [5]:
#define special functions used in excel file

#return a number rounded to the desired multiple
def mround(a, b):
    return (abs(a % b) >= abs(b / 2))*b + (int(a / b))*b

#return a/b; otherwise capture errors when calculating and returns 0
def iferror(a,b, t = 0):
    try:
        return a/b
    except:
        return t

def con_ratios(x):
    #First, calculate ratio helpers
    #Since ratio helpers are not calculated in order, list every formula instead of using loop
    
    #Define as1 correspends to column AS in excel spreedsheet. 
    #The element as1[i] correspends column AS row i in excel spreedsheet.Other empty rows are 0.
    
    as1 = np.zeros(56)
    as1[10] = x.dot(characteristics['CET1'].values)
    as1[11] = x.dot(characteristics['Tier 1 \ncapital'].values)
    as1[12] = x.dot(characteristics['Total capital \nstd'].values)
    as1[13] = x.dot(characteristics['Total capital\nadv'].values)
    as1[14] = x.dot(characteristics['TLAC'].values)
    as1[15] = x.dot(characteristics['LTD'].values)
    as1[16] = x.dot(characteristics['Std \nRWA'].values)
    as1[17] = x.dot(characteristics['Adv \nRWA'].values)
    as1[18] = x.dot(characteristics['CCAR \nStd RWA'].values)
    as1[19] = x.dot(characteristics['Spot\nGAAP \nassets'].values)
    as1[20] = x.dot(characteristics['Spot GAAP \nliabilities'].values)
    as1[21] = x.dot(characteristics['Basel III leverage'].values)
    as1[22] = x.dot(characteristics['CCAR leverage'].values)
    as1[31] = x.dot(characteristics['SLR'].values) * GSIB_coefficient['US'].iloc[0]
    as1[29] = x.dot(characteristics['Cross-JD \nclaims'].values) * GSIB_coefficient['US'].iloc[1]
    as1[30] = x.dot(characteristics['Cross-JD \nliabilities'].values) * GSIB_coefficient['US'].iloc[2]
    as1[32] = x.dot(characteristics['IFS \nassets'].values) * GSIB_coefficient['US'].iloc[3]
    as1[33] = x.dot(characteristics['IFS \nliabilities'].values) * GSIB_coefficient['US'].iloc[4]
    as1[34] = x.dot(characteristics['Marketable \nsecurities'].values) * GSIB_coefficient['US'].iloc[5]
    as1[35] = x.dot(characteristics['OTC \nderivs'].values) * GSIB_coefficient['US'].iloc[6]
    as1[36] = x.dot(characteristics['L3 \nassets'].values) * GSIB_coefficient['US'].iloc[7]
    as1[37] = x.dot(characteristics['Trad / \nAFS \nsecurities'].values) * GSIB_coefficient['US'].iloc[8]
    as1[38] = x.dot(characteristics['STWF']) / max(as1[16], as1[17]) * 350
    as1[39] = x.dot(characteristics['SLR'].values) * GSIB_coefficient['Basel'].iloc[0]
    as1[40] = x.dot(characteristics['Cross-JD \nclaims'].values) * GSIB_coefficient['Basel'].iloc[1]
    as1[41] = x.dot(characteristics['Cross-JD \nliabilities'].values) * GSIB_coefficient['Basel'].iloc[2]
    as1[42] = x.dot(characteristics['IFS \nassets'].values) * GSIB_coefficient['Basel'].iloc[3]
    as1[43] = x.dot(characteristics['IFS \nliabilities'].values) * GSIB_coefficient['Basel'].iloc[4]
    as1[44] = x.dot(characteristics['Marketable \nsecurities'].values) * GSIB_coefficient['Basel'].iloc[5]
    as1[45] = x.dot(characteristics['OTC \nderivs'].values) * GSIB_coefficient['Basel'].iloc[6]
    as1[46] = x.dot(characteristics['L3 \nassets'].values) * GSIB_coefficient['Basel'].iloc[7]
    as1[47] = x.dot(characteristics['Trad / \nAFS \nsecurities'].values) * GSIB_coefficient['Basel'].iloc[8]
    as1[48] = 100
    as1[49] = x.dot(characteristics['CCAR losses, actual'].values)*(-1)
    as1[50] = x.dot(characteristics['JPM\nLAB'].values)
    as1[51] = x.dot(characteristics['JPM\nNFO'].values)
    as1[52] = x.dot(characteristics['LCR\nHQLA'].values)
    as1[53] = x.dot(characteristics['LCR\nNFO'].values)
    as1[54] = x.dot(characteristics['ASF'].values)
    as1[55] = x.dot(characteristics['RSF'].values)

    as1[28] = as1[39] + as1[40] + as1[41] + as1[42] + as1[43] + as1[44] + as1[45] + as1[46] + as1[47] + as1[48]
    as1[27] = as1[28] / 2e4
    as1[26] = mround((as1[28] + 20) / 1e4, .01) / 2
    as1[25] = as1[29] + as1[30] + as1[31] + as1[32] + as1[33] + as1[34] + as1[35] + as1[36] + as1[37] + as1[38]
    as1[24] = as1[25] / 2e4
    as1[23] = mround((as1[25] + 20) / 2e4, .005)
    as1[4] = as1[24] * as1[16]
    as1[5] = as1[24] * as1[17]
    as1[6] = as1[24] * as1[18]
    as1[7] = as1[26] * as1[16]
    as1[8] = as1[26] * as1[17]
    as1[9] = as1[26] * as1[18]
    
    
    #Second, calculate constrainted ratios
    
    #Define ao1 correspends to column AO in excel spreedsheet. 
    #The element ao1[i] correspends column AO row i in excel spreedsheet.Other empty rows are 0.     
    
    ao1 = np.zeros(33)
    
    ao1[5] = iferror((as1[10] - as1[4]), as1[16])
    ao1[6] = iferror((as1[11] - as1[4]), as1[16])
    ao1[7] = iferror((as1[12] - as1[4]), as1[16])
    ao1[8] = iferror((as1[14] - as1[4]), as1[16])
    ao1[9] = iferror((as1[15] - as1[4]), as1[16])
    ao1[10] = iferror((as1[10] - as1[5]), as1[17])
    ao1[11] = iferror((as1[11] - as1[5]), as1[17])
    ao1[12] = iferror((as1[13] - as1[5]), as1[17])
    ao1[13] = iferror((as1[14] - as1[8]), as1[17])
    ao1[14] = iferror((as1[15] - as1[5]), as1[17])
    ao1[15] = iferror((as1[10] - as1[49]), as1[18])
    ao1[16] = iferror((as1[11] - as1[49]), as1[18])
    ao1[17] = iferror((as1[12] - as1[49]), as1[18])
    ao1[18] = iferror(as1[11], as1[21])
    ao1[19] = iferror(as1[14], as1[21])
    ao1[20] = iferror(as1[15], as1[21])
    ao1[21] = iferror((as1[11] - as1[49]), as1[22])
    ao1[24] = iferror(as1[50], as1[51])
    ao1[25] = iferror(as1[52], as1[53])
    ao1[26] = iferror(as1[54], as1[55])
    ao1[29] = ao1[30] = as1[25]
    ao1[31] = ao1[32] = as1[28]


    #delete 0 and generate an array with 24 target ratios stored 
    index = np.where(ao1 == 0)#get indexs of 0
    new_ao1 = np.delete(ao1, index) # delete 0
    #delete NaN and generate an array with 24 target ratios' thresholds stored
    threshold = constraint_ratio['Constraint'].dropna().values    
    
    
    #Use output array to record constraints status. 
    #If constrint holds, record 1, otherwise 0. 
    output = np.array([])
    
    # n1 = 24 inequality on ratio constraints
    for i in range(n1):
        if new_ao1[i] >= threshold[i]:
            output = np.append(output,1)
        else:
            output = np.append(output,0)
    # 1 equality on GAAP asst and GAAP liability
    if as1[19] == as1[20]:
        output = np.append(output,1)
    else:
        output = np.append(output,0) 
    
    #if all cnstraints are satisfied, return 0; otherwise -1
    return np.prod(output)-1


<font face="Arial" size=4 color=#404040 > 4.3 Constraint 3: LINKAGES</font>
<font face="Arial" size=3 color=#404040>
    
    * There are n2 = 89 constraints in total
    * For every column: (optimized notional - starting notional) >= sum of every row[(optimized notional - starting notional) * correlation number]
    * To simplify codes, record constraints status into an array. If constraint holds, record 1, otherwise 0. Only when all contraints holds, the product of the array equals to 1, otherwise 0.

In [6]:
def con_linkage(x):
    
    #define a dictionary called idmap: the key i is Product name and the value is the product index in DataFrame characteristics
    idmap = {i: n for n, i in enumerate(characteristics['Product'])}
    output = np.array([])
    
    #for every column, the total change of the product itself >= sum of changes coming from changes of all rows 
    for i in linkages.columns[1:]:
        tmp_linkage = linkages[i]
        
        #the total change of the product itself 
        deltas = x[idmap[i]] - characteristics[characteristics['Product'] == i]['Starting notional'].iloc[0]
        #sum of changes coming from changes of all rows 
        for n, j in enumerate(linkages.iloc[:, 0]):
            deltas -= (x[idmap[j]] - characteristics[characteristics['Product'] == j]['Starting notional'].iloc[0]) * tmp_linkage.iloc[n]
        
        # n2 = 89 inequality on ratio constraints
        if deltas >= 0:
            output = np.append(output,1)
        else:
            output = np.append(output,0)
    
    #if all cnstraints are satisfied, return 0, else return -1
    return np.prod(output)-1

#connect constraints together and store in a dictionary called cons
con1 = {'type': 'eq', 'fun': con_ratios}
con2 = {'type': 'eq', 'fun': con_linkage} 
cons = ([con1,con2]) 

<font face="Arial" size=5 color=#00000 > 5.Initial Projection</font>
<font face="Arial" size=3 color=#404040>
    
    * Use starting notional as initial projection

In [7]:
x0 = np.empty(N)
for i in range(N):
    x0[i] = characteristics.iloc[i,1]

<font face="Arial" size=5 color=#000000 > 6.Result</font>

In [8]:
result = minimize(objective,x0,method='SLSQP',bounds=bounds_capacity,constraints=cons)
sva = result.fun*(-1)
opti = result.x 

print('the maximized sva is {}'.format(sva))
print('the calculation exits successfully: {}'.format(result.success))
print('the result message: {}'.format(result.message))
print('the optimized notional is {}'.format(opti))


the maximized sva is 174929.52148746757
the calculation exits successfully: False
the result message: Singular matrix C in LSQ subproblem
the optimized notional is [6.90015760e+00 2.91668313e+00 2.80936071e+00 7.98955180e+00
 1.06771249e+01 3.01381910e+00 3.78768224e+00 1.19451915e+00
 1.31129513e+00 3.32664728e+01 3.88496677e+01 0.00000000e+00
 1.26364999e-03 6.91156255e-01 3.59933364e-01 1.50496828e-01
 6.95678633e-02 1.44414875e+01 6.18965418e+00 8.17150644e-02
 3.01450062e-02 1.77031957e-01 7.95888684e-02 8.96022930e+00
 3.34167792e+00 3.62446890e+00 3.79749543e+00 6.52326843e+00
 1.07149855e+00 2.00326030e+00 7.44587518e+00 1.97344057e+01
 3.43811080e+00 1.72886659e+00 1.91385946e+00 2.00690380e+01
 1.67000580e+01 1.56994560e+01 0.00000000e+00 0.00000000e+00
 1.21514306e-02 2.51776054e+00 7.13351389e-01 1.54767013e+01
 4.93671972e+00 1.47077204e-01 1.99394539e+01 1.19480321e+01
 1.10681840e+01 5.76223496e+00 8.99736912e+00 1.00144668e+01
 6.30820527e+00 2.11242021e+01 1.45296772e+

<font face="Arial" size=5 color=#000000 > 7.Examine
<font face="Arial" size=3 color=#404040>
    
    * Capacity
    * Ratio
    * Linkage

In [9]:
#examine if capacity constraints hold
diff1 = (opti >= lower) & (opti >= 0)
diff2 = (opti <= upper) | (opti <= 0) 
diff3 = opti >= 0
examine1 = np.prod(diff1) * np.prod(diff2) * np.prod(diff3)

if examine1 ==1:
    print('the capacity constraints hold')
else:
    print('Error')

the capacity constraints hold


In [10]:
#examine if ratio constraints hold
examine2 = con_ratios(opti)

if examine2 ==0:
    print('the ratio constraints hold')
else:
    print('Error')

Error


In [11]:
#examine if linkage constraints hold
examine3 = con_linkage(opti)

if examine3 ==0:
    print('the linkage constraints hold')
else:
    print('Error')

the linkage constraints hold


<font face="Arial" size=5 color=#000000 > 8.Export result</font>

In [12]:
# Bulid a workbook
file = xlwt.Workbook(encoding='utf-8', style_compression=0) 
sheet = file.add_sheet('test', cell_overwrite_ok=True) 

header = ['result']
data=opti

# store data
sheet.write(0,0,'Optimized notional')
for i in range(N):
    sheet.write(i+1,0,data[i])

# save data into excel file named result
file.save(r'e:\result.xls')


<font face="Arial" size=5 color=#000000> 9. Constraint Relaxation</font>

In [13]:
# No consideration of capacity
result_new = minimize(objective,x0,method='SLSQP',bounds=None,constraints=cons)
sva_new = result_new.fun*(-1)
opti_new = result_new.x 

print('the maximized sva is {}'.format(sva_new))
print('the calculation exits successfully: {}'.format(result_new.success))
print('the result message: {}'.format(result_new.message))
print('the optimized notional is {}'.format(opti_new))

the maximized sva is 195459.6206594144
the calculation exits successfully: False
the result message: Singular matrix C in LSQ subproblem
the optimized notional is [1.64689235e+01 2.91668313e+00 2.80936071e+00 7.98955180e+00
 1.06771249e+01 3.01381910e+00 3.78768224e+00 1.19451915e+00
 1.31129513e+00 3.32664728e+01 3.88496677e+01 5.17279940e-01
 2.41294583e-01 6.91156255e-01 3.59933364e-01 1.50496828e-01
 6.95678633e-02 1.44414875e+01 6.18965418e+00 8.17150644e-02
 3.01450062e-02 1.77031957e-01 7.95888684e-02 8.96022930e+00
 3.34167792e+00 3.62446890e+00 3.79749543e+00 2.54764752e+01
 7.98306312e+00 4.65531694e+00 1.46728269e+01 1.97344057e+01
 3.43811080e+00 1.72886659e+00 1.91385946e+00 2.00690380e+01
 1.67000580e+01 1.56994560e+01 0.00000000e+00 0.00000000e+00
 1.21514306e-02 2.51776054e+00 7.13351389e-01 1.54767013e+01
 4.93671972e+00 1.47077204e-01 2.69923104e+01 5.99949063e+00
 1.06912036e+01 3.35780332e+00 2.13514366e+01 9.90113507e+00
 6.25656691e+00 1.60809211e+01 1.45296772e+0